In [1]:
import sys
import numpy as np
import pickle
import os
import glob
import networkx as nx
from tqdm import tqdm
import random

sys.path.append('../../')
from DataConstructor.notebooks.leafer import Leafer



seed = 42
random.seed(seed)
np.random.seed(seed)

In [2]:
G = nx.read_edgelist("../../TaxonomyEnrichment/data/verb/train.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [3]:
wn = True

while True:
    try:
        cycle = nx.find_cycle(G)
        print(cycle)
        G.remove_edge(*cycle[0])
    except:
        break

if not wn:
    new_labels = {}
    for node in G.nodes():
        new_labels[node] = node + '.n.1'

    G_new = nx.relabel_nodes(G, new_labels)

    l = Leafer(G_new)
else:
    l = Leafer(G)

In [4]:
len(G.edges())

11682

In [5]:
train, test = l.split_train_test(
    generation_depth=0,  # до какого уровня в топ. сортировке идти
    p=0.0,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.5,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.5,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.00, 0.0, 0.0, 0.00, 0.00, 1.],
    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка, предсказание родителя
    #p_parent=1
    use_1sense_hypernyms=False
)

predict_hypernym 7822 7822


In [6]:
test_path = '../data/verb/test_nodes.pickle'
with open(test_path, 'rb') as f:
    test_nodes = pickle.load(f)

In [7]:
test_nodes[:5], train[:5]

([('hit.v.15', ['consume.v.02']),
  ('submit.v.06', ['yield.v.13']),
  ('shaft.v.01', ['equip.v.01']),
  ('concuss.v.01', ['shake.v.01']),
  ('raise.v.21', ['incite.v.02'])],
 [{'children': 'lock_out.v.01',
   'parents': 'exclude.v.02',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'caddie.v.01',
   'parents': 'serve.v.02',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'fort.v.01',
   'parents': 'meet.v.07',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'overshadow.v.03',
   'parents': 'obscure.v.01',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'chariot.v.01',
   'parents': 'transport.v.02',
   'grandparents': None,
   'case': 'predict_hypernym'}])

In [8]:
test = []

for child, parents in test_nodes:
    elem = {}
    elem["children"] = child
    elem["grandparents"] = None
    if len(parents) == 1:
        elem["parents"] = parents[0]
        elem["case"] = "predict_hypernym"
    else:
        elem["parents"] = parents
        elem["case"] = "predict_multiple_hypernyms"

    test.append(elem)

In [9]:
test[:5]

[{'children': 'hit.v.15',
  'grandparents': None,
  'parents': 'consume.v.02',
  'case': 'predict_hypernym'},
 {'children': 'submit.v.06',
  'grandparents': None,
  'parents': 'yield.v.13',
  'case': 'predict_hypernym'},
 {'children': 'shaft.v.01',
  'grandparents': None,
  'parents': 'equip.v.01',
  'case': 'predict_hypernym'},
 {'children': 'concuss.v.01',
  'grandparents': None,
  'parents': 'shake.v.01',
  'case': 'predict_hypernym'},
 {'children': 'raise.v.21',
  'grandparents': None,
  'parents': 'incite.v.02',
  'case': 'predict_hypernym'}]

In [10]:
train[:5]

[{'children': 'lock_out.v.01',
  'parents': 'exclude.v.02',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'caddie.v.01',
  'parents': 'serve.v.02',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'fort.v.01',
  'parents': 'meet.v.07',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'overshadow.v.03',
  'parents': 'obscure.v.01',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'chariot.v.01',
  'parents': 'transport.v.02',
  'grandparents': None,
  'case': 'predict_hypernym'}]

In [31]:
with open('../data/verb/test_hypernyms.pickle', 'wb') as f:
    pickle.dump(test, f)

with open('../data/verb/train_hypernyms_multisense.pickle', 'wb') as f:
    pickle.dump(train, f)